# Week 2: Geostrophy and thermal wind balance
### MAQ - 32806, Chiel van Heerwaarden & Imme Benedict, 2016
In this assignment you will investigate whether the large-scale flow is in geostrophic and thermal wind balance. For this, you will use ECMWF ERA-Interim data from the 1st of January 2016 0:00.
___

### Setting up a Python Basemap and read out 3D fields
We start the tutorial by loading the required Python packages and setting the figure properties.

In [ ]:
import numpy as np              # Numpy is the fundamental package for scientific computing in Python.
import netCDF4 as nc            # NetCDF is the data format of the meteorological data that we use.
import matplotlib.pyplot as pl  # Matplotlib is a scientific plotting package.
import matplotlib.colors as cl
from mpl_toolkits.basemap import Basemap # Import the map plotting interface.
import warnings
warnings.filterwarnings('ignore')

# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%matplotlib inline
pl.rcParams.update({'font.size': 11})          # Set the standard font size of the plots to 11pt.
pl.rcParams.update({'figure.figsize': [13,6]}) # Set the standard figure size.

___
With the code below, you initialize the world map with the desired coordinates.

In [ ]:
# We define a map of the northern hemisphere that is stored in the m object.
lon_start = - 90.
lon_end   =   90.
lat_start =   10.
lat_end   =   65.
m = Basemap(llcrnrlon=lon_start,
            llcrnrlat=lat_start,
            urcrnrlon=lon_end,
            urcrnrlat=lat_end,
            projection='mill', resolution='l')

___
Now, you load the data and read out the desired variables from a NetCDF file from the ECMWF ERA-Interim data archive. You are loading a file with data from October 1st, 2015.

In [ ]:
nc_file = nc.Dataset("data/era_data.nc", "r")
#nc_file = nc.Dataset("data/era5_data.nc", "r")
lat = nc_file.variables["latitude"][:]
lon = nc_file.variables["longitude"][:]
p = nc_file.variables["level"][:]*100.
t = 0
nqstep = 3
nroll = lon.size//2
lon = np.roll(lon, nroll)
lon = np.where(lon>=180., lon-360., lon)

u = np.roll(nc_file.variables["u"][t,:,:,:], nroll, -1)
v = np.roll(nc_file.variables["v"][t,:,:,:], nroll, -1)
z = np.roll(nc_file.variables["z"][t,:,:,:], nroll, -1)
T = np.roll(nc_file.variables["t"][t,:,:,:], nroll, -1)

lons, lats = m(*np.meshgrid(lon, lat))
lonrad, latrad = np.meshgrid(np.pi/180.*lon, np.pi/180.*lat)

___
### Plotting the temperature and geopotential on both maps
As a starting point you find below the temperature and geopotential at 500 hPa.

In [ ]:
p_plot = 50000.
n = abs(p-p_plot).argmin()
nq = nqstep # In the quiver plot we take steps of nq, otherwise there are too many arrows.

m.contourf(lons, lats, T[n,:,:], 20, cmap=pl.cm.jet)
m.colorbar()
qu = m.quiver(lons[::nq,::nq], lats[::nq,::nq],
              u[n,::nq,::nq], v[n,::nq, ::nq],
              pivot='mid', width=1.2e-3,
              scale=1000.)
    
cz = m.contour(lons, lats, z[n,:,:], 15, colors='w', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')
m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-80.,90,20.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,60.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('T and Z at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

___
### Assignment
In this assignment you are going to verify the existence of geostrophic wind balance and thermal wind balance. The instructions below give you some hints how to proceed. You are free to plot everything that you consider helpful in this task. Work out the question in the large map or choose a region that you find interesting (thus change the map region in the beginning of the script. Make sure to restart the notebook via the `Kernel` item on top when you change the map range).

#### Geostrophic balance
1. Explain the meaning of geostrophic wind balance and write it out in pressure coordinates. Does the flow in the plot seem to be in geostrophic balance?
1. Calculate and plot the geostrophic wind speed $\left| \boldsymbol{V_g} \right|$ at 500 hPa and compare it with the actual wind speed $\left| \boldsymbol{V} \right|$. This is a complex task that contains a sequence of steps. You have to calculate the geostrophic wind components $u_g$ and $v_g$ from the 3d field `z` that contains the geopotential $\Phi$. You need the Coriolis parameter $f \equiv 2\Omega \sin \phi$ as well. This one you have to calculate yourself.
2. Where is the atmosphere in geostrophic balance and where not? Discuss the explanation in detail.
3. Do the same at an other height of your choice and discuss the differences with the 500 hPa level.

#### Thermal wind balance
1. Derive the thermal wind balance in pressure coordinates. Explain the meaning in detail.
2. Calculate and plot the term that contains the vertical gradient (around 500 hPa) of the zonal geostrophic wind.
3. Calculate and plot the term that contains the meridional gradient of the temperature.
4. Is the atmosphere in thermal wind balance? Where is it and where not?


___
Work out the assignment below. If you feel more comfortable in using Word, you can also work the assignment out in there. You can save the pictures by clicking right on them. You can use the Notebook from last week to look up how to calculate gradients.
___

In [ ]:
omega = 7.292e-5
a = 6.37e6
g = 9.81
R = 287.

In [ ]:
dzdx = 1./(a * np.cos(latrad)) * np.gradient(z, axis=2) / np.gradient(lonrad, axis=1)
dzdy = 1./a * np.gradient(z, axis=1) / np.gradient(latrad, axis=0)
f = 2.*omega*np.sin(latrad)
ug = -(1./f)*dzdy
vg =  (1./f)*dzdx
Vg = (ug**2 + vg**2)**.5
V = (u**2 + v**2)**.5

In [ ]:
p_plot = 50000.
n = abs(p-p_plot).argmin()
nq = nqstep

V_color_range = np.arange(0, 60, 2.5)

pl.figure()
m.contourf(lons, lats, V[n,:,:], V_color_range, cmap=pl.cm.jet, extend='both')
m.colorbar()
qu = m.quiver(lons[::nq,::nq], lats[::nq,::nq],
              u[n,::nq,::nq], v[n,::nq, ::nq],
              pivot='mid', width=1.2e-3,
              scale=1000.)
    
cz = m.contour(lons, lats, z[n,:,:], 15, colors='w', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')
m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-80.,90,20.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,60.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('V and Z at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

pl.figure()
m.contourf(lons, lats, Vg[n,:,:], V_color_range, cmap=pl.cm.jet, extend='both')
m.colorbar()
qu = m.quiver(lons[::nq,::nq], lats[::nq,::nq],
              u[n,::nq,::nq], v[n,::nq, ::nq],
              pivot='mid', width=1.2e-3,
              scale=1500.)
    
cz = m.contour(lons, lats, z[n,:,:], 15, colors='w', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')
m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-80.,90,20.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,60.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('Vg and Z at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

V_diff_range = np.linspace(-25., 25., 21)

pl.figure()
m.contourf(lons, lats, V[n,:,:]-Vg[n,:,:], V_diff_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()
qu = m.quiver(lons[::nq,::nq], lats[::nq,::nq],
              u[n,::nq,::nq], v[n,::nq, ::nq],
              pivot='mid', width=1.2e-3,
              scale=1500.)
    
cz = m.contour(lons, lats, z[n,:,:], 15, colors='k', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')
m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-80.,90,20.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,60.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('Va and Z at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

In [ ]:
lat_plot = 40.
lon_plot = -60.
i_lon = abs(lon - lon_plot).argmin()
j_lat = abs(lat - lat_plot).argmin()

pl.subplot(121)
pl.plot(T[:,j_lat,i_lon], p, 'bo-')
pl.xlabel('T (K)')
pl.ylabel('p (Pa)')
pl.gca().invert_yaxis()

pl.subplot(122)
pl.plot(V [:,j_lat,i_lon], p, 'r^:' , label='U')
pl.plot(Vg[:,j_lat,i_lon], p, 'm*--', label='Ug')
pl.xlabel('wind speed (m/s)')
pl.ylabel('p (Pa)')
pl.legend(loc=0, frameon=False)
pl.gca().invert_yaxis()
pl.tight_layout()


In [ ]:
dTdx = 1./(a * np.cos(latrad)) * np.gradient(T, axis=2) / np.gradient(lonrad, axis=1)
dTdy = 1./a * np.gradient(T, axis=1) / np.gradient(latrad, axis=0)
dugdlnp = np.gradient(ug, axis=0) / np.gradient(np.log(p[:, np.newaxis, np.newaxis]), axis=0)
dvgdlnp = np.gradient(vg, axis=0) / np.gradient(np.log(p[:, np.newaxis, np.newaxis]), axis=0)

In [ ]:
p_plot = 50000.
n = abs(p-p_plot).argmin()
nq = nqstep # In the quiver plot we take steps of nq, otherwise there are too many arrows.

ut_range = np.linspace(-60., 60., 21)

pl.figure()
m.contourf(lons, lats, R/f*dTdy[n,:,:], ut_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()
qu = m.quiver(lons[::nq,::nq], lats[::nq,::nq],
              u[n,::nq,::nq], v[n,::nq, ::nq],
              pivot='mid', width=1.2e-3,
              scale=1500.)
    
cz = m.contour(lons, lats, z[n,:,:], 15, colors='w', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')
m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-80.,90,20.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,60.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('R/f dTdy at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

pl.figure()
m.contourf(lons, lats, dugdlnp[n,:,:], ut_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()
qu = m.quiver(lons[::nq,::nq], lats[::nq,::nq],
              u[n,::nq,::nq], v[n,::nq, ::nq],
              pivot='mid', width=1.2e-3,
              scale=1500.)
    
cz = m.contour(lons, lats, z[n,:,:], 15, colors='w', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')
m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-80.,90,20.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,60.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('dug/dlnp at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

pl.figure()
m.contourf(lons, lats, R/f*dTdy[n,:,:]-dugdlnp[n,:,:], ut_range, cmap=pl.cm.seismic, extend='both')
m.colorbar()
qu = m.quiver(lons[::nq,::nq], lats[::nq,::nq],
              u[n,::nq,::nq], v[n,::nq, ::nq],
              pivot='mid', width=1.2e-3,
              scale=1500.)
    
cz = m.contour(lons, lats, z[n,:,:], 15, colors='w', linewidths=1.5)
pl.clabel(cz, fontsize=10., fmt='%1.0f')
m.drawcoastlines(color='#666666')
# draw parallels.
parallels = np.arange(-80.,90,20.)
m.drawparallels(parallels,labels=[1,0,0,0])
# draw meridians
meridians = np.arange(0.,360.,60.)
m.drawmeridians(meridians,labels=[0,0,0,1])
pl.title('dvgdlnp at p = {0:.0f} hPa'.format(p[n] / 100));
pl.tight_layout()

In [ ]:
pl.contourf(lat, p, T[:,:,lon_plot], 21)
pl.colorbar()

V_range = np.arange(-50, 100, 10)
cV = pl.contour(lat, p, u[:,:,lon_plot], V_range, colors='k', linewidths=1.)
pl.clabel(cV, fontsize=10., fmt='%1.0f')
pl.gca().invert_yaxis()
pl.xlabel('lat')
pl.ylabel('p (Pa)')
pl.ylim(100000, 1000)
pl.tight_layout()

In [ ]:
lat_plot = 40.
lon_plot = -60.
i_lon = abs(lon - lon_plot).argmin()
j_lat = abs(lat - lat_plot).argmin()

pl.subplot(121)
pl.plot(1000*dTdy[:,j_lat,i_lon], p, 'bo-')
pl.xlabel('dTdy (K/km)')
pl.ylabel('p (Pa)')
pl.gca().invert_yaxis()

pl.subplot(122)
pl.plot(dugdlnp[:,j_lat,i_lon], p, 'r^-' , label='dug/dlnp')
pl.xlabel('dug/dlnp')
pl.ylabel('p (Pa)')
pl.legend(loc=0, frameon=False)
pl.gca().invert_yaxis()
pl.tight_layout()

In [ ]:
pl.figure()
pl.contourf(lat, p, T[:,:,lon_plot], 21)
pl.colorbar()
pl.contour(lat, p, T[:,:,lon_plot], 21, colors='k')
pl.gca().invert_yaxis()
pl.xlabel('lat')
pl.ylabel('p (Pa)')
pl.ylim(100000, 1000)
pl.title('Pressure coordinates')
pl.tight_layout()

latp, pp = np.meshgrid(lat, p)

pl.figure()
pl.contourf(latp, z[:,:,lon_plot]/9.81, T[:,:,lon_plot], 21)
pl.colorbar()
pl.contour(latp, z[:,:,lon_plot]/9.81, T[:,:,lon_plot], 21, colors='k')
pl.clabel(cV, fontsize=10., fmt='%1.0f')
pl.xlabel('lat')
pl.ylabel('z (m)')
pl.ylim(0, 25000)
pl.title('Height coordinates')
pl.tight_layout()